In [3]:
import pandas as pd
import numpy as np

# 2 Mean-Variance Optimization

In [4]:
df = pd.read_excel("../data/multi_asset_etf_data.xlsx", sheet_name="excess returns")
df.set_index("Date", inplace=True)

## 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:
$$\text{sharpe ratio of investment }i = \frac{\mux_i}{\sigma_i}$$

In [5]:
def performance_stat(s: pd.Series) -> pd.Series:
    """
    Calculate the mean, volatility, sharpe of given series
    
    Parameters:
        s (pd.Series): 
            Excess return of certain asset / portfolio
            Index: all time period (i.e. monthly)
            
    Returns:
        s_stat: Series contains mean, volatility, sharpe of the input series
    """
    s_stat = s.agg(['mean', 'std']).T
    s_stat['mean'] *= 12
    s_stat['std'] *= (12 ** (1/2))
    s_stat['sharpe'] = s_stat['mean'] / s_stat['std']
    return s_stat

In [6]:
df_stat = df.apply(performance_stat).T
df_stat.sort_values('sharpe', inplace=True)
print(f"Best performer: {df_stat.index[-1]}; Worse performer: {df_stat.index[0]}")
df_stat

Best performer: SPY; Worse performer: BWX


,mean,std,sharpe
BWX,-0.001843,0.083359,-0.022112
DBC,0.025443,0.178975,0.142162
IEF,0.014269,0.062405,0.228652
EEM,0.064887,0.196531,0.330163
PSP,0.079938,0.227387,0.351552
QAI,0.018974,0.050810,0.373440
TIP,0.022321,0.051529,0.433166
EFA,0.081597,0.165991,0.491573
IYR,0.129473,0.187101,0.691997
HYG,0.064168,0.089154,0.719746


## 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

In [7]:
df_corr = df.corr()
df_corr_unstack = df_corr.unstack().sort_values()
df_corr_unstack = df_corr_unstack[df_corr_unstack < 1]
print(f"Highest correlation: {df_corr_unstack.index[-1]}; Lowest correlation: {df_corr_unstack.index[0]}")
df_corr

Highest correlation: ('PSP', 'SPY'); Lowest correlation: ('DBC', 'IEF')


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.000000,0.349773,0.647614,0.621662,0.557653,0.434472,0.453534,0.524870,0.668045,0.465713,0.617099
DBC,0.349773,1.000000,0.565654,0.581865,0.473208,-0.321738,0.318314,0.496057,0.547936,0.509886,0.136668
EEM,0.647614,0.565654,1.000000,0.851579,0.726041,-0.102347,0.621814,0.771677,0.807245,0.734556,0.302729
EFA,0.621662,0.581865,0.851579,1.000000,0.771463,-0.132331,0.697875,0.891929,0.853674,0.871641,0.287476
HYG,0.557653,0.473208,0.726041,0.771463,1.000000,-0.008598,0.757649,0.823823,0.768756,0.770353,0.365939
IEF,0.434472,-0.321738,-0.102347,-0.132331,-0.008598,1.000000,0.073622,-0.118676,0.055667,-0.155696,0.706078
IYR,0.453534,0.318314,0.621814,0.697875,0.757649,0.073622,1.000000,0.760158,0.655963,0.753610,0.397166
PSP,0.524870,0.496057,0.771677,0.891929,0.823823,-0.118676,0.760158,1.000000,0.838287,0.895729,0.320913
QAI,0.668045,0.547936,0.807245,0.853674,0.768756,0.055667,0.655963,0.838287,1.000000,0.840989,0.459712
SPY,0.465713,0.509886,0.734556,0.871641,0.770353,-0.155696,0.753610,0.895729,0.840989,1.000000,0.294639


In [8]:
print("By table 1, TIP has higher mean return and sharpe ratio than IEF (domestic bonds) and BWX (foreign bonds).")

By table 1, TIP has higher mean return and sharpe ratio than IEF (domestic bonds) and BWX (foreign bonds).


## 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\wtan$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\wtan$.

In [9]:
def mv_frontier_tang(df: pd.DataFrame, is_reg: bool = False) -> pd.Series:
    """
    Calculate the tangent portfolio weight on efficient frontier
    
    Parameters:
        df (pd.DataFrame): 
            Excess return df.
            Columns: all assets
            Index: all time period (i.e. monthly)
        is_reg (bool): 
            If True, use regularized covariance matrix.
            
    Returns:
        weight_tang: tangent portfolio weight in ascending order
    """
    df_cov_ann = df.cov()
    if is_reg:
        df_cov_ann += np.diag(np.diag(df_cov_ann))
        df_cov_ann /= 2
    df_cov_mean = pd.Series(np.linalg.inv(df_cov_ann) @ df.mean(), 
                            index=df_cov_ann.columns.to_list(),
                            name='Tangency Weight')
    df_tang = df_cov_mean / df_cov_mean.sum()
    df_tang.sort_values(inplace=True)
    return df_tang

In [10]:
weight_tang = mv_frontier_tang(df)
weight_tang

QAI   -3.133445
BWX   -1.464974
PSP   -1.271055
IYR   -0.242772
DBC    0.028436
EEM    0.261028
TIP    0.356935
EFA    0.452914
HYG    1.528942
IEF    1.893992
SPY    2.589999
Name: Tangency Weight, dtype: float64

In [11]:
print(f"The ranking of weight of tangency portfolio "
      f"{'=' if all(df_stat.index == weight_tang.index) else '!='} sharpe ratio rank.")

The ranking of weight of tangency portfolio != sharpe ratio rank.


In [12]:
df_tang = (df * weight_tang.loc[df.columns]).sum(axis=1)
df_tang_stat = performance_stat(df_tang)
df_tang_stat

mean      0.370180
std       0.191523
sharpe    1.932824
dtype: float64

## 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

In [13]:
# a) without TIPS
weight_tang_excl_tips = mv_frontier_tang(df.drop(columns=["TIP"]))
df_tang_excl_tips = (df * weight_tang_excl_tips.loc[df.drop(columns=["TIP"]).columns]).sum(axis=1)
performance_stat(df_tang_excl_tips)

mean      0.386291
std       0.200111
sharpe    1.930383
dtype: float64

In [14]:
# Change of tangent portfolio
weight_tang_excl_tips - weight_tang

BWX   -0.047776
DBC    0.026721
EEM    0.017058
EFA   -0.011418
HYG    0.064197
IEF    0.318459
IYR   -0.003123
PSP   -0.043037
QAI   -0.105510
SPY    0.141364
TIP         NaN
Name: Tangency Weight, dtype: float64

In [15]:
# a) TIPS += 0.0012
df_tip_adj = df.copy()
df_tip_adj["TIP"] += 0.0012
weight_tang_adj_tips = mv_frontier_tang(df_tip_adj)
df_tang_adj_tips = (df_tip_adj * weight_tang_adj_tips.loc[df_tip_adj.columns]).sum(axis=1)
performance_stat(df_tang_adj_tips)

mean      0.328908
std       0.161947
sharpe    2.030967
dtype: float64

In [16]:
# Change of tangent portfolio
weight_tang_adj_tips - weight_tang

BWX    0.202111
DBC   -0.113042
EEM   -0.072163
EFA    0.048303
HYG   -0.271582
IEF   -1.347217
IYR    0.013213
PSP    0.182065
QAI    0.446354
SPY   -0.598029
TIP    1.509987
Name: Tangency Weight, dtype: float64

In [17]:
print("""
Based on the results, TIPS should be not considered as a separate assets
1. sharpe ratio of tangent portfolio with/without TIPS is almost the same (with-TIPS has marginal advantage);
2. when TIPS increase overall, tangent portfolio has a higher sharpe
""")


Based on the results, TIPS should be not considered as a separate assets
1. sharpe ratio of tangent portfolio with/without TIPS is almost the same (with-TIPS has marginal advantage);
2. when TIPS increase overall, tangent portfolio has a higher sharpe


# 3. Allocations

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\mutarg$. Thus, the $i$ element of the weight vector is,
$$\wEW_i = \frac{1}{n}$$

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,
$$\wRP_i = \frac{1}{\sigma_i^2}$$

#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 
$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$
but this time, use a regularized covariance matrix,
$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values. 

### Comparing

In order to compare all these allocation methods, (those above, along with the tangency portfolio obtained in the previous section,) rescale each weight vector, such that it has targeted mean return of $\mutarg$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How do these compare across the four allocation methods?